<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/model_evaluation/mlflow_bleu_metric_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLflow BLEU Metric Demonstration

This notebook provides a comprehensive guide to using MLflow's BLEU (Bilingual Evaluation Understudy) metric for evaluating machine translation and text generation models.

## What is BLEU?

BLEU is a metric for automatically evaluating machine-translated text by comparing it to one or more reference translations. Key characteristics:

- **Precision-based**: Measures how many n-grams in the candidate appear in the reference
- **Brevity Penalty**: Penalizes translations that are too short
- **N-gram Matching**: Typically uses 1-grams through 4-grams
- **Corpus-level**: Designed for corpus-level evaluation but can be adapted for sentence-level

### BLEU Score Range:
- 0.0 to 1.0 (often reported as 0-100)
- Higher scores indicate better translation quality
- Human translations typically score 0.6-0.8
- Good MT systems: 0.3-0.5

## 1. Installation and Setup

In [ ]:
# Install required packages
!pip install -q mlflow nltk sacrebleu pandas numpy matplotlib seaborn plotly

In [ ]:
import mlflow
import mlflow.metrics
from mlflow.metrics import make_metric, MetricValue
import pandas as pd
import numpy as np
import json
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# BLEU implementations
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
import sacrebleu

# Download required NLTK data
nltk.download('punkt', quiet=True)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print(f"MLflow version: {mlflow.__version__}")
print(f"SacreBLEU version: {sacrebleu.__version__}")

## 2. Creating Custom BLEU Metrics for MLflow

In [ ]:
class BLEUMetrics:
    """
    Comprehensive BLEU metric implementations for MLflow.
    """

    @staticmethod
    def create_bleu_metric(n_gram=4, smoothing=True, use_sacrebleu=True):
        """
        Create a BLEU metric for MLflow evaluation.

        Args:
            n_gram: Maximum n-gram order (typically 4)
            smoothing: Apply smoothing for sentence-level BLEU
            use_sacrebleu: Use SacreBLEU (more standardized) vs NLTK
        """
        def bleu_metric(predictions, targets):
            if not isinstance(predictions, list):
                predictions = [predictions]
            if not isinstance(targets, list):
                targets = [targets]

            scores = []

            if use_sacrebleu:
                # SacreBLEU implementation
                for pred, target in zip(predictions, targets):
                    # SacreBLEU expects references as list
                    score = sacrebleu.sentence_bleu(pred, [target])
                    scores.append(score.score / 100.0)  # Normalize to 0-1
            else:
                # NLTK implementation
                smoothing_fn = SmoothingFunction().method1 if smoothing else None

                for pred, target in zip(predictions, targets):
                    # Tokenize if strings
                    if isinstance(pred, str):
                        pred = pred.split()
                    if isinstance(target, str):
                        target = target.split()

                    weights = tuple([1/n_gram] * n_gram)
                    score = sentence_bleu(
                        [target], pred,
                        weights=weights,
                        smoothing_function=smoothing_fn
                    )
                    scores.append(score)

            return MetricValue(
                aggregate_results={f"bleu_{n_gram}": np.mean(scores)},
                scores=scores
            )

        return make_metric(
            eval_fn=bleu_metric,
            greater_is_better=True,
            name=f"bleu_{n_gram}_{'sacre' if use_sacrebleu else 'nltk'}"
        )

    @staticmethod
    def create_multi_bleu_metric():
        """
        Create a metric that computes BLEU-1 through BLEU-4.
        """
        def multi_bleu_metric(predictions, targets):
            if not isinstance(predictions, list):
                predictions = [predictions]
            if not isinstance(targets, list):
                targets = [targets]

            all_scores = {f"bleu_{i}": [] for i in range(1, 5)}

            for pred, target in zip(predictions, targets):
                # Calculate BLEU-1 through BLEU-4
                for n in range(1, 5):
                    score = sacrebleu.sentence_bleu(
                        pred, [target],
                        max_ngram_order=n
                    )
                    all_scores[f"bleu_{n}"].append(score.score / 100.0)

            aggregate_results = {
                key: np.mean(values)
                for key, values in all_scores.items()
            }

            # Add geometric mean (similar to BLEU-4 but explicit)
            aggregate_results["bleu_geometric_mean"] = np.exp(
                np.mean([np.log(aggregate_results[f"bleu_{i}"]) for i in range(1, 5)])
            )

            return MetricValue(
                aggregate_results=aggregate_results,
                scores=all_scores["bleu_4"]  # Return BLEU-4 as primary score
            )

        return make_metric(
            eval_fn=multi_bleu_metric,
            greater_is_better=True,
            name="multi_bleu"
        )

# Create metric instances
bleu_4_sacre = BLEUMetrics.create_bleu_metric(n_gram=4, use_sacrebleu=True)
bleu_4_nltk = BLEUMetrics.create_bleu_metric(n_gram=4, use_sacrebleu=False)
multi_bleu = BLEUMetrics.create_multi_bleu_metric()

print("✅ Custom BLEU metrics created successfully!")

## 3. Preparing Sample Translation Data

In [ ]:
# Create diverse translation examples
translation_data = [
    {
        "source": "The weather is beautiful today.",
        "reference": "Das Wetter ist heute schön.",
        "model_good": "Das Wetter ist heute schön.",
        "model_medium": "Das Wetter ist schön heute.",
        "model_poor": "Wetter gut heute."
    },
    {
        "source": "Machine learning revolutionizes data analysis.",
        "reference": "Maschinelles Lernen revolutioniert die Datenanalyse.",
        "model_good": "Maschinelles Lernen revolutioniert die Datenanalyse.",
        "model_medium": "Machine Learning revolutioniert Datenanalyse.",
        "model_poor": "Computer lernen Daten."
    },
    {
        "source": "The cat sits on the comfortable mat.",
        "reference": "Die Katze sitzt auf der bequemen Matte.",
        "model_good": "Die Katze sitzt auf der gemütlichen Matte.",
        "model_medium": "Die Katze ist auf der Matte.",
        "model_poor": "Katze Matte."
    },
    {
        "source": "Artificial intelligence transforms modern businesses.",
        "reference": "Künstliche Intelligenz transformiert moderne Unternehmen.",
        "model_good": "Künstliche Intelligenz verwandelt moderne Unternehmen.",
        "model_medium": "KI transformiert Unternehmen.",
        "model_poor": "Computer ändern Geschäft."
    },
    {
        "source": "The quick brown fox jumps over the lazy dog.",
        "reference": "Der schnelle braune Fuchs springt über den faulen Hund.",
        "model_good": "Der schnelle braune Fuchs springt über den trägen Hund.",
        "model_medium": "Der braune Fuchs springt über den Hund.",
        "model_poor": "Fuchs springt Hund."
    }
]

# Convert to DataFrame
df_translations = pd.DataFrame(translation_data)

print(f"📊 Created {len(df_translations)} translation examples")
print("\nExample translations:")
for idx, row in df_translations.head(2).iterrows():
    print(f"\n{idx+1}. Source: {row['source']}")
    print(f"   Reference: {row['reference']}")
    print(f"   Good Model: {row['model_good']}")
    print(f"   Poor Model: {row['model_poor']}")

## 4. Evaluating Translation Models with MLflow

In [ ]:
# Set up MLflow experiment
mlflow.set_experiment("bleu-metrics-demo")

def evaluate_translation_model(model_name, predictions, references, model_config=None):
    """
    Comprehensive evaluation of a translation model using BLEU metrics.
    """
    with mlflow.start_run(run_name=model_name):
        # Log model configuration
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("num_samples", len(predictions))

        if model_config:
            for key, value in model_config.items():
                mlflow.log_param(key, value)

        # Create evaluation dataframe
        eval_df = pd.DataFrame({
            "predictions": predictions,
            "targets": references
        })

        # Evaluate with BLEU metrics
        metrics = [bleu_4_sacre, multi_bleu]

        results = mlflow.evaluate(
            data=eval_df,
            targets="targets",
            predictions="predictions",
            extra_metrics=metrics,
            evaluators="default"
        )

        # Calculate additional detailed metrics
        detailed_scores = calculate_detailed_bleu(predictions, references)

        # Log detailed metrics
        for key, value in detailed_scores.items():
            mlflow.log_metric(key, value)

        return results.metrics, detailed_scores

def calculate_detailed_bleu(predictions, references):
    """
    Calculate detailed BLEU scores including n-gram precisions.
    """
    scores = {}

    # Calculate individual n-gram scores
    for n in range(1, 5):
        bleu_scores = []
        for pred, ref in zip(predictions, references):
            score = sacrebleu.sentence_bleu(pred, [ref], max_ngram_order=n)
            bleu_scores.append(score.score / 100.0)

        scores[f"bleu_{n}_mean"] = np.mean(bleu_scores)
        scores[f"bleu_{n}_std"] = np.std(bleu_scores)
        scores[f"bleu_{n}_min"] = np.min(bleu_scores)
        scores[f"bleu_{n}_max"] = np.max(bleu_scores)

    # Calculate corpus-level BLEU
    corpus_score = sacrebleu.corpus_bleu(predictions, [references])
    scores["corpus_bleu"] = corpus_score.score / 100.0

    # Add brevity penalty info
    scores["brevity_penalty"] = corpus_score.bp

    return scores

# Evaluate all three models
models_config = {
    "High-Quality NMT": {
        "predictions": df_translations["model_good"].tolist(),
        "config": {"architecture": "transformer", "params": "60M", "training_data": "10M_pairs"}
    },
    "Medium-Quality NMT": {
        "predictions": df_translations["model_medium"].tolist(),
        "config": {"architecture": "lstm", "params": "10M", "training_data": "1M_pairs"}
    },
    "Low-Quality Baseline": {
        "predictions": df_translations["model_poor"].tolist(),
        "config": {"architecture": "word-by-word", "params": "1M", "training_data": "100K_pairs"}
    }
}

evaluation_results = {}
for model_name, config in models_config.items():
    metrics, detailed = evaluate_translation_model(
        model_name,
        config["predictions"],
        df_translations["reference"].tolist(),
        config["config"]
    )
    evaluation_results[model_name] = {"metrics": metrics, "detailed": detailed}

print("✅ Evaluation completed for all models!")

## 5. Comprehensive Results Analysis

In [ ]:
# Create comprehensive comparison table
comparison_data = []

for model_name, results in evaluation_results.items():
    row = {"Model": model_name}

    # Add main BLEU scores
    for n in range(1, 5):
        key = f"bleu_{n}_mean"
        if key in results["detailed"]:
            row[f"BLEU-{n}"] = results["detailed"][key]

    # Add corpus BLEU and brevity penalty
    row["Corpus BLEU"] = results["detailed"].get("corpus_bleu", 0)
    row["Brevity Penalty"] = results["detailed"].get("brevity_penalty", 0)

    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

# Display formatted results
print("\n📊 BLEU Score Comparison (0-1 scale):")
print("="*80)

# Format for display
display_df = comparison_df.copy()
for col in display_df.columns:
    if col != "Model":
        display_df[col] = display_df[col].apply(lambda x: f"{x:.4f}")

print(display_df.to_string(index=False))

# Calculate improvements
print("\n📈 Performance Analysis:")
print("="*80)

baseline_bleu4 = comparison_df[comparison_df["Model"] == "Low-Quality Baseline"]["BLEU-4"].values[0]
for _, row in comparison_df.iterrows():
    if row["Model"] != "Low-Quality Baseline":
        improvement = ((row["BLEU-4"] - baseline_bleu4) / baseline_bleu4) * 100
        print(f"{row['Model']:25s}: {improvement:+.1f}% improvement over baseline")

## 6. Advanced Visualizations

In [ ]:
# Create interactive visualizations with Plotly
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "BLEU Scores by N-gram Order",
        "Model Performance Radar Chart",
        "Score Distribution",
        "Sentence-Level BLEU Heatmap"
    ),
    specs=[
        [{"type": "bar"}, {"type": "scatterpolar"}],
        [{"type": "box"}, {"type": "heatmap"}]
    ]
)

colors = {
    "High-Quality NMT": "#2ecc71",
    "Medium-Quality NMT": "#f39c12",
    "Low-Quality Baseline": "#e74c3c"
}

# Plot 1: BLEU Scores by N-gram
for model_name in comparison_df["Model"]:
    row_data = comparison_df[comparison_df["Model"] == model_name]
    bleu_scores = [row_data[f"BLEU-{n}"].values[0] for n in range(1, 5)]

    fig.add_trace(
        go.Bar(
            name=model_name,
            x=["BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4"],
            y=bleu_scores,
            marker_color=colors[model_name]
        ),
        row=1, col=1
    )

# Plot 2: Radar Chart
for model_name in comparison_df["Model"]:
    row_data = comparison_df[comparison_df["Model"] == model_name]

    fig.add_trace(
        go.Scatterpolar(
            r=[
                row_data["BLEU-1"].values[0],
                row_data["BLEU-2"].values[0],
                row_data["BLEU-3"].values[0],
                row_data["BLEU-4"].values[0],
                row_data["Brevity Penalty"].values[0]
            ],
            theta=["BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4", "Brevity\nPenalty"],
            fill='toself',
            name=model_name,
            marker_color=colors[model_name]
        ),
        row=1, col=2
    )

# Plot 3: Score Distribution (Box Plot)
for model_name, config in models_config.items():
    # Calculate sentence-level BLEU-4 scores
    sentence_scores = []
    for pred, ref in zip(config["predictions"], df_translations["reference"].tolist()):
        score = sacrebleu.sentence_bleu(pred, [ref])
        sentence_scores.append(score.score / 100.0)

    fig.add_trace(
        go.Box(
            y=sentence_scores,
            name=model_name,
            marker_color=colors[model_name]
        ),
        row=2, col=1
    )

# Plot 4: Heatmap of sentence-level scores
heatmap_data = []
for model_name, config in models_config.items():
    sentence_scores = []
    for pred, ref in zip(config["predictions"], df_translations["reference"].tolist()):
        score = sacrebleu.sentence_bleu(pred, [ref])
        sentence_scores.append(score.score / 100.0)
    heatmap_data.append(sentence_scores)

fig.add_trace(
    go.Heatmap(
        z=heatmap_data,
        x=[f"Sent {i+1}" for i in range(len(df_translations))],
        y=["High-Quality", "Medium", "Low-Quality"],
        colorscale="RdYlGn",
        text=np.round(heatmap_data, 3),
        texttemplate="%{text}",
        textfont={"size": 10}
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text="Comprehensive BLEU Metric Analysis",
    showlegend=True,
    height=800,
    width=1200
)

fig.update_xaxes(title_text="N-gram Order", row=1, col=1)
fig.update_yaxes(title_text="BLEU Score", row=1, col=1)
fig.update_yaxes(title_text="BLEU-4 Score", row=2, col=1)

fig.show()

## 7. Production-Ready BLEU Evaluation Pipeline

In [ ]:
class BLEUEvaluationPipeline:
    """
    Production-ready BLEU evaluation pipeline with MLflow integration.
    """

    def __init__(self, experiment_name="bleu-evaluation", tracking_uri=None):
        self.experiment_name = experiment_name
        if tracking_uri:
            mlflow.set_tracking_uri(tracking_uri)
        mlflow.set_experiment(experiment_name)

        # Initialize metrics
        self.metrics = {
            "bleu_4_sacre": BLEUMetrics.create_bleu_metric(4, use_sacrebleu=True),
            "bleu_4_nltk": BLEUMetrics.create_bleu_metric(4, use_sacrebleu=False),
            "multi_bleu": BLEUMetrics.create_multi_bleu_metric()
        }

    def evaluate_model(self,
                       model_name: str,
                       predictions: List[str],
                       references: List[str],
                       metadata: Dict = None) -> Dict:
        """
        Evaluate a single model with comprehensive BLEU metrics.
        """
        with mlflow.start_run(run_name=model_name):
            # Log metadata
            mlflow.log_param("model_name", model_name)
            mlflow.log_param("num_samples", len(predictions))

            if metadata:
                for key, value in metadata.items():
                    mlflow.log_param(key, value)

            # Calculate all BLEU variants
            results = self._calculate_comprehensive_bleu(predictions, references)

            # Log metrics
            for metric_name, metric_value in results["aggregate"].items():
                mlflow.log_metric(metric_name, metric_value)

            # Log artifacts
            self._log_evaluation_artifacts(predictions, references, results)

            return results

    def _calculate_comprehensive_bleu(self,
                                     predictions: List[str],
                                     references: List[str]) -> Dict:
        """
        Calculate comprehensive BLEU metrics.
        """
        results = {
            "sentence_level": [],
            "aggregate": {},
            "n_gram_precision": {}
        }

        # Sentence-level scores
        for pred, ref in zip(predictions, references):
            sentence_metrics = {}

            # Calculate BLEU-1 through BLEU-4
            for n in range(1, 5):
                score = sacrebleu.sentence_bleu(pred, [ref], max_ngram_order=n)
                sentence_metrics[f"bleu_{n}"] = score.score / 100.0

            # Add detailed scores
            full_score = sacrebleu.sentence_bleu(pred, [ref])
            sentence_metrics["brevity_penalty"] = full_score.bp
            sentence_metrics["length_ratio"] = full_score.sys_len / full_score.ref_len

            results["sentence_level"].append(sentence_metrics)

        # Aggregate scores
        for metric_name in results["sentence_level"][0].keys():
            values = [s[metric_name] for s in results["sentence_level"]]
            results["aggregate"][f"{metric_name}_mean"] = np.mean(values)
            results["aggregate"][f"{metric_name}_std"] = np.std(values)
            results["aggregate"][f"{metric_name}_min"] = np.min(values)
            results["aggregate"][f"{metric_name}_max"] = np.max(values)

        # Corpus-level BLEU
        corpus_score = sacrebleu.corpus_bleu(predictions, [references])
        results["aggregate"]["corpus_bleu"] = corpus_score.score / 100.0
        results["aggregate"]["corpus_brevity_penalty"] = corpus_score.bp

        # N-gram precision breakdown
        for n in range(1, 5):
            corpus_n = sacrebleu.corpus_bleu(
                predictions, [references],
                max_ngram_order=n
            )
            results["n_gram_precision"][f"precision_{n}_gram"] = corpus_n.precisions[n-1] / 100.0

        return results

    def _log_evaluation_artifacts(self,
                                 predictions: List[str],
                                 references: List[str],
                                 results: Dict):
        """
        Log evaluation artifacts to MLflow.
        """
        # Create evaluation report
        report = {
            "summary": results["aggregate"],
            "n_gram_precision": results["n_gram_precision"],
            "sample_analysis": [
                {
                    "prediction": pred,
                    "reference": ref,
                    "scores": scores
                }
                for pred, ref, scores in zip(
                    predictions[:5],  # Log first 5 samples
                    references[:5],
                    results["sentence_level"][:5]
                )
            ]
        }

        mlflow.log_dict(report, "evaluation_report.json")

    def compare_models(self,
                      models: Dict[str, Tuple[List[str], Dict]],
                      references: List[str]) -> pd.DataFrame:
        """
        Compare multiple models.

        Args:
            models: Dict of model_name -> (predictions, metadata)
            references: Reference translations
        """
        comparison_results = []

        for model_name, (predictions, metadata) in models.items():
            results = self.evaluate_model(
                model_name, predictions, references, metadata
            )

            # Extract key metrics for comparison
            row = {
                "Model": model_name,
                "BLEU-1": results["aggregate"]["bleu_1_mean"],
                "BLEU-2": results["aggregate"]["bleu_2_mean"],
                "BLEU-3": results["aggregate"]["bleu_3_mean"],
                "BLEU-4": results["aggregate"]["bleu_4_mean"],
                "Corpus BLEU": results["aggregate"]["corpus_bleu"],
                "Avg Length Ratio": results["aggregate"]["length_ratio_mean"]
            }
            comparison_results.append(row)

        return pd.DataFrame(comparison_results)

# Demonstrate the pipeline
pipeline = BLEUEvaluationPipeline(experiment_name="bleu-production-pipeline")

# Prepare models for comparison
models_to_compare = {
    "Transformer-Large": (
        df_translations["model_good"].tolist(),
        {"architecture": "transformer", "params": "175M", "beam_size": 5}
    ),
    "LSTM-Attention": (
        df_translations["model_medium"].tolist(),
        {"architecture": "lstm_attention", "params": "50M", "beam_size": 3}
    ),
    "Statistical-MT": (
        df_translations["model_poor"].tolist(),
        {"architecture": "phrase_based", "params": "10M", "beam_size": 1}
    )
}

# Run comparison
comparison_results = pipeline.compare_models(
    models_to_compare,
    df_translations["reference"].tolist()
)

print("\n🚀 Production Pipeline Results:")
print("="*80)
print(comparison_results.round(4).to_string(index=False))

# Identify best model
best_model = comparison_results.loc[comparison_results["BLEU-4"].idxmax()]
print(f"\n🏆 Best Model: {best_model['Model']}")
print(f"   - BLEU-4: {best_model['BLEU-4']:.4f}")
print(f"   - Corpus BLEU: {best_model['Corpus BLEU']:.4f}")

## 8. Statistical Significance Testing

In [ ]:
def bootstrap_bleu_significance(predictions_a, predictions_b, references,
                                n_bootstrap=1000, confidence=0.95):
    """
    Perform bootstrap resampling to test statistical significance of BLEU differences.
    """
    n_samples = len(predictions_a)

    # Calculate original BLEU scores
    score_a = sacrebleu.corpus_bleu(predictions_a, [references]).score / 100.0
    score_b = sacrebleu.corpus_bleu(predictions_b, [references]).score / 100.0
    original_diff = score_a - score_b

    # Bootstrap resampling
    bootstrap_diffs = []

    for _ in range(n_bootstrap):
        # Sample with replacement
        indices = np.random.choice(n_samples, n_samples, replace=True)

        boot_preds_a = [predictions_a[i] for i in indices]
        boot_preds_b = [predictions_b[i] for i in indices]
        boot_refs = [references[i] for i in indices]

        boot_score_a = sacrebleu.corpus_bleu(boot_preds_a, [boot_refs]).score / 100.0
        boot_score_b = sacrebleu.corpus_bleu(boot_preds_b, [boot_refs]).score / 100.0

        bootstrap_diffs.append(boot_score_a - boot_score_b)

    # Calculate confidence interval
    alpha = 1 - confidence
    lower_percentile = (alpha/2) * 100
    upper_percentile = (1 - alpha/2) * 100

    ci_lower = np.percentile(bootstrap_diffs, lower_percentile)
    ci_upper = np.percentile(bootstrap_diffs, upper_percentile)

    # Check if significant (CI doesn't include 0)
    is_significant = (ci_lower > 0) or (ci_upper < 0)

    return {
        "model_a_bleu": score_a,
        "model_b_bleu": score_b,
        "difference": original_diff,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper,
        "is_significant": is_significant,
        "p_value_estimate": np.mean([d <= 0 for d in bootstrap_diffs]) * 2  # Two-tailed
    }

# Test significance between models
print("\n📊 Statistical Significance Testing (Bootstrap)")
print("="*80)

# Compare High-Quality vs Medium-Quality
sig_test_1 = bootstrap_bleu_significance(
    df_translations["model_good"].tolist(),
    df_translations["model_medium"].tolist(),
    df_translations["reference"].tolist()
)

print("\nHigh-Quality NMT vs Medium-Quality NMT:")
print(f"  High-Quality BLEU: {sig_test_1['model_a_bleu']:.4f}")
print(f"  Medium-Quality BLEU: {sig_test_1['model_b_bleu']:.4f}")
print(f"  Difference: {sig_test_1['difference']:.4f}")
print(f"  95% CI: [{sig_test_1['ci_lower']:.4f}, {sig_test_1['ci_upper']:.4f}]")
print(f"  Significant: {'Yes ✓' if sig_test_1['is_significant'] else 'No ✗'}")
print(f"  P-value (estimate): {sig_test_1['p_value_estimate']:.4f}")

# Compare Medium-Quality vs Low-Quality
sig_test_2 = bootstrap_bleu_significance(
    df_translations["model_medium"].tolist(),
    df_translations["model_poor"].tolist(),
    df_translations["reference"].tolist()
)

print("\nMedium-Quality NMT vs Low-Quality Baseline:")
print(f"  Medium-Quality BLEU: {sig_test_2['model_a_bleu']:.4f}")
print(f"  Low-Quality BLEU: {sig_test_2['model_b_bleu']:.4f}")
print(f"  Difference: {sig_test_2['difference']:.4f}")
print(f"  95% CI: [{sig_test_2['ci_lower']:.4f}, {sig_test_2['ci_upper']:.4f}]")
print(f"  Significant: {'Yes ✓' if sig_test_2['is_significant'] else 'No ✗'}")
print(f"  P-value (estimate): {sig_test_2['p_value_estimate']:.4f}")

## 9. Multi-Reference BLEU Evaluation

In [ ]:
# Create multi-reference data
multi_ref_data = [
    {
        "source": "The weather is beautiful today.",
        "references": [
            "Das Wetter ist heute schön.",
            "Heute ist das Wetter wunderschön.",
            "Es ist schönes Wetter heute."
        ],
        "prediction": "Das Wetter ist heute schön."
    },
    {
        "source": "Machine learning revolutionizes data analysis.",
        "references": [
            "Maschinelles Lernen revolutioniert die Datenanalyse.",
            "Machine Learning verändert die Datenanalyse grundlegend.",
            "Datenanalyse wird durch maschinelles Lernen revolutioniert."
        ],
        "prediction": "Maschinelles Lernen revolutioniert die Datenanalyse."
    }
]

def evaluate_multi_reference_bleu(predictions, multi_references):
    """
    Evaluate BLEU with multiple references per source.
    """
    scores = []

    for pred, refs in zip(predictions, multi_references):
        # SacreBLEU handles multiple references naturally
        score = sacrebleu.sentence_bleu(pred, refs)
        scores.append(score.score / 100.0)

    # Calculate corpus-level score with multiple references
    # Transpose references for corpus_bleu format
    refs_transposed = list(map(list, zip(*multi_references)))
    corpus_score = sacrebleu.corpus_bleu(predictions, refs_transposed)

    return {
        "sentence_scores": scores,
        "average_score": np.mean(scores),
        "corpus_score": corpus_score.score / 100.0
    }

# Evaluate with multi-reference
predictions = [item["prediction"] for item in multi_ref_data]
multi_refs = [item["references"] for item in multi_ref_data]

multi_ref_results = evaluate_multi_reference_bleu(predictions, multi_refs)

print("\n🎯 Multi-Reference BLEU Evaluation:")
print("="*60)
print(f"Average Sentence BLEU: {multi_ref_results['average_score']:.4f}")
print(f"Corpus BLEU: {multi_ref_results['corpus_score']:.4f}")
print("\nSentence-level scores:")
for i, score in enumerate(multi_ref_results['sentence_scores']):
    print(f"  Sentence {i+1}: {score:.4f}")

print("\n💡 Note: Multi-reference BLEU typically yields higher scores")
print("   as it credits translations matching any reference variant.")

## 10. Best Practices and Recommendations

### Key Takeaways:

1. **BLEU Metric Selection**:
   - **BLEU-4**: Standard for machine translation evaluation
   - **BLEU-1**: Good for content word coverage
   - **Multi-BLEU**: Provides comprehensive n-gram analysis
   - **SacreBLEU**: Use for standardized, reproducible scores

2. **Evaluation Considerations**:
   - Always report corpus-level BLEU for overall system performance
   - Use sentence-level BLEU with smoothing for fine-grained analysis
   - Consider brevity penalty impact on short translations
   - Use multiple references when available

3. **Statistical Significance**:
   - Always test significance when comparing models
   - Use bootstrap resampling for confidence intervals
   - Report p-values alongside BLEU scores

4. **MLflow Integration Benefits**:
   - Automatic experiment tracking and versioning
   - Easy A/B testing and model comparison
   - Reproducible evaluation pipelines
   - Centralized metric storage

5. **Production Tips**:
   - Cache tokenization for large-scale evaluation
   - Use parallel processing for sentence-level scores
   - Implement early stopping based on BLEU thresholds
   - Monitor BLEU drift in production

### BLEU Score Interpretation:
- **< 0.1**: Poor quality, barely usable
- **0.1-0.2**: Low quality, major improvements needed
- **0.2-0.3**: Reasonable quality for some applications
- **0.3-0.4**: Good quality for production MT systems
- **0.4-0.5**: Very good quality, approaching human performance
- **> 0.5**: Excellent quality, often human-level

In [ ]:
# Final MLflow tracking summary
print("\n📈 MLflow Tracking Summary:")
print("="*60)
print("To view all experiments and metrics in MLflow UI:")
print("\n1. Run in terminal:")
print("   mlflow ui --port 5000")
print("\n2. Open browser:")
print("   http://localhost:5000")
print("\n3. Navigate to experiments:")
print("   - bleu-metrics-demo")
print("   - bleu-production-pipeline")
print("\n✅ Demo completed successfully!")
print("\n🔗 Additional Resources:")
print("   - MLflow Docs: https://mlflow.org/docs/latest/")
print("   - SacreBLEU: https://github.com/mjpost/sacrebleu")
print("   - BLEU Paper: https://aclanthology.org/P02-1040/")